In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.mlab as mlab

#### 匯入會計、產業以及KAM檔案並做整理和merge

Acc_path = './2020Account_0720.xlsx' #'./2020Account_0720.xlsx'
Account = pd.read_excel(Acc_path)
Account = pd.DataFrame(Account)

Ind_path = './2020IndustryCode_0802.xlsx'  #'./2020IndustryCode_0802.xlsx'
Industry = pd.read_excel(Ind_path)
Industry = pd.DataFrame(Industry)

KAM_path = './2020關鍵查核事項_1129.xlsx'  #'./2020ALLKAM_DF0624.xlsx'
KAM = pd.read_excel(KAM_path)
KAM = pd.DataFrame(KAM)

In [44]:
Acc_path = './2021Account_1201.xlsx' #'./2020Account_0720.xlsx'
Account = pd.read_excel(Acc_path)
Account = pd.DataFrame(Account)

Ind_path = './2021IndustryCode_0829.xlsx'  #'./2020IndustryCode_0802.xlsx'
Industry = pd.read_excel(Ind_path)
Industry = pd.DataFrame(Industry)

KAM_path = './2021關鍵查核事項_1130.xlsx'  #'./2020ALLKAM_DF0624.xlsx'
KAM = pd.read_excel(KAM_path)
KAM = pd.DataFrame(KAM)

In [45]:
#刪掉沒有會計師事務所的公司
Account.dropna(subset=['事務所碼', '事務所名稱'],inplace = True)
Account.isnull().sum()#for check

Unnamed: 0       0
公司代碼             0
公司名稱             0
年度               0
序號               0
母公司總資產           0
母公司淨營收           0
合併總資產            0
合併淨營收            0
產業代碼             0
產業名稱             0
揭露類別             0
揭露條件Y         1451
揭露條件A         2029
揭露條件B         1928
自願揭露            25
審計公費級距           0
非審計公費級距          0
會計師公費級距          0
公司審計公費           0
審計公費含子公司         0
公司非審計公費          0
公司非審－制度設計        0
公司非審－工商登記        0
事務所碼             0
事務所名稱            0
會計師一             0
會計師二             0
會計師三          1850
會計師四          2011
dtype: int64

Acc_Ind = pd.merge(Industry,Account, on='公司代碼', how='inner')
Acc_Ind = Acc_Ind.drop(columns=['Unnamed: 0_x','Unnamed: 0_y','公司名稱_y','年/月', '季別', '合併(Y/N)', '月份', '會計師三', '會計師四'])
Acc_Ind.rename(columns={'公司名稱_x':'公司名稱'}, inplace=True)
Acc_Ind.columns

In [46]:
DF = pd.merge(Account,KAM, on='公司名稱', how='inner')#(Acc_Ind,KAM, on='公司代碼', how='inner')
DF = DF.drop(columns=['Unnamed: 0_x','Unnamed: 0_y','公司代碼_y','年月'])

DF.rename(columns={'公司代碼_x':'公司代碼'}, inplace=True)
DF.columns

Index(['公司代碼', '公司名稱', '年度', '序號', '母公司總資產', '母公司淨營收', '合併總資產', '合併淨營收',
       '產業代碼', '產業名稱', '揭露類別', '揭露條件Y', '揭露條件A', '揭露條件B', '自願揭露', '審計公費級距',
       '非審計公費級距', '會計師公費級距', '公司審計公費', '審計公費含子公司', '公司非審計公費', '公司非審－制度設計',
       '公司非審－工商登記', '事務所碼', '事務所名稱', '會計師一', '會計師二', '會計師三', '會計師四', 'KAM'],
      dtype='object')

In [47]:
#檢查重複公司
accc = DF[['公司名稱']]
accc[accc.duplicated()]

,公司名稱


#### 計算資料筆數

In [48]:
print('Account_Ori:',len(Account))
print('IndustryCode_Ori:',len(Industry))
print('KAM_Ori:',len(KAM))
print('DF:',len(DF))

Account_Ori: 2042
IndustryCode_Ori: 2171
KAM_Ori: 2144
DF: 2025


#### 加入是否為四大會計事務所col

In [49]:
#加入判斷是否為四大會計事務所欄位
ap = []
af = DF[["事務所碼"]]
for (index, colname) in enumerate(af): #遍歷 enumerate
    f = af[colname].values #遍歷方式 取a欄值
    
for i in range(len(f)):
    if f[i] == 'KK152': #安侯建業聯合
        boo = 1
        ap.append(boo)
    elif f[i] == 'KK158':  #安永聯合
        boo = 1
        ap.append(boo)
    elif f[i] == 'KK203':  #勤業眾信聯合
        boo = 1
        ap.append(boo)
    elif f[i] == 'KK149':  #資誠聯合
        boo = 1
        ap.append(boo)
    else:
        boo = 0
        ap.append(boo)


DF["是否為四大事務所"] = ap

#### 加入四大會計事務所標籤col

In [51]:
#判斷四大會計事務所1-4
ju = []
af = DF[["事務所碼"]]
for (index, colname) in enumerate(af):
    f = af[colname].values
for i in range(len(f)):
    if f[i] == 'KK149': #資誠聯合
        boo = 1
        ju.append(boo)
    elif f[i] == 'KK152':  #安侯建業聯合
        boo = 2
        ju.append(boo)
    elif f[i] == 'KK158':  #安永聯合
        boo = 3
        ju.append(boo)
    elif f[i] == 'KK203':  #勤業眾信聯合
        boo = 4
        ju.append(boo)
    else:
        boo = 0
        ju.append(boo)


DF["四大事務所編碼"] = ju

#### OneHotEncoder 加上產業編號

#print(DF['交易所主產業代碼'].nunique())
#print(DF['交易所主產業名稱'].value_counts())

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

le = LabelEncoder()
DF['交易所主產業標籤'] = le.fit_transform(DF['交易所主產業代碼'])
DF[['公司名稱', '交易所主產業代碼', '交易所主產業標籤']].sample(5)

#### KAM分析

In [52]:
DF.columns

Index(['公司代碼', '公司名稱', '年度', '序號', '母公司總資產', '母公司淨營收', '合併總資產', '合併淨營收',
       '產業代碼', '產業名稱', '揭露類別', '揭露條件Y', '揭露條件A', '揭露條件B', '自願揭露', '審計公費級距',
       '非審計公費級距', '會計師公費級距', '公司審計公費', '審計公費含子公司', '公司非審計公費', '公司非審－制度設計',
       '公司非審－工商登記', '事務所碼', '事務所名稱', '會計師一', '會計師二', '會計師三', '會計師四', 'KAM',
       '是否為四大事務所', '四大事務所編碼'],
      dtype='object')

In [53]:
#取出需要欄位
KAM_df = DF[['公司代碼', '公司名稱', '事務所碼', '事務所名稱', "是否為四大事務所", "四大事務所編碼", '會計師一', '會計師二', 'KAM']]
#'交易所主產業標籤','交易所主產業代碼', '交易所主產業名稱', '交易所子產業代碼', '交易所子產業名稱', 產業分類先拿掉

#### KAM情感分析

In [54]:
from ckiptagger import WS
#匯入中文情感字典

stl = []
with open('ST_Litigious.txt','r',encoding="utf-8") as fh:
    tmp = fh.read()
    onelinelist = tmp.split('\n')
    for i in onelinelist:
        stl.append(i)

stms = []
with open('ST_ModalStrong.txt','r',encoding="utf-8") as fh:
    tmp = fh.read()
    onelinelist = tmp.split('\n')
    for i in onelinelist:
        stms.append(i)
        
stn = []
with open('ST_Negative.txt','r',encoding="utf-8") as fh:
    tmp = fh.read()
    onelinelist = tmp.split('\n')
    for i in onelinelist:
        stn.append(i)
        
stmw = []
with open('ST_ModalWeak.txt','r',encoding="utf-8") as fh:
    tmp = fh.read()
    onelinelist = tmp.split('\n')
    for i in onelinelist:
        stmw.append(i)
        
stp = []
with open('ST_Positive.txt','r',encoding="utf-8") as fh:
    tmp = fh.read()
    onelinelist = tmp.split('\n')
    for i in onelinelist:
        stp.append(i)
        
stuc = []
with open('ST_Uncertainty.txt','r',encoding="utf-8") as fh:
    tmp = fh.read()
    onelinelist = tmp.split('\n')
    for i in onelinelist:
        stuc.append(i)

In [55]:
#將KAM用CKIPTAGGER做斷詞並存入out list

t1 =[]
out = []
for k in range(len(DF)):
    #print(DF['KAM'][k])
    t1.append(DF['KAM'][k])

ws = WS("./data")

out = ws(t1) #以out[0]可取得第0個KAM斷詞過後的結果



C:\Users\user\anaconda3\lib\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


In [56]:
cn_stops = ['：','，','、','(',')','；','.','。','!','?','"','*','/','-','_','#','@','$','&','<','>','=','[',']','＼','^',
            '{','}','|','~','｢','｣','〃','》','『','』','】','【','〕','〔','〟','〖','〗','〘','〙','〚','〛','〝','〞','｀','〰','──']
tagg = []
last = []
for i in range(len(out)):
    word_list = out[i]
    for word in word_list: 
        if word not in cn_stops:
            last.append(word)
    tagg.append(last)
    last = []

word_len = []
for l in range(len(tagg)):
    word_len.append(len(tagg[l]))

In [57]:
lstl = []
lstms = []
lstmw = []
lstp = []
lstn = []
lstuc = []
for i in range(len(out)):
    l = 0
    ms = 0
    mw = 0
    p = 0
    n = 0
    uc = 0
    for j in out[i]:
        if j in stl:  #j是被斷詞過後財報裡每一個詞 
            l = l+1  #若財報裡的詞存在於stl字典計數加一
       
        if j in stms:  
            ms = ms+1         
         
        if j in stmw:  
            mw = mw+1         
        
        if j in stp:  
            p = p+1         
        
        if j in stn:  
            n = n+1         
        
        if j in stuc:  
            uc = uc+1         
        
        else:
            pass
        
    lstl.append(l) #把每一筆KAM的stl字詞數量append到list stl裡面最後在一次加回到dataframe
    lstms.append(ms)
    lstmw.append(mw)
    lstp.append(p)
    lstn.append(n)
    lstuc.append(uc)

In [58]:
KAM_df['KAM_len'] = word_len #詞數

KAM_df["Litigious"] = lstl
KAM_df["ModalStrong"] = lstms
KAM_df["ModalWeak"] = lstmw
KAM_df["Negative"] = lstn
KAM_df["Positive"] = lstp
KAM_df["Uncertainty"] = lstuc

<ipython-input-58-721954fea6d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  KAM_df['KAM_len'] = word_len #詞數
<ipython-input-58-721954fea6d9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  KAM_df["Litigious"] = lstl
<ipython-input-58-721954fea6d9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [59]:
KAM_df.columns

Index(['公司代碼', '公司名稱', '事務所碼', '事務所名稱', '是否為四大事務所', '四大事務所編碼', '會計師一', '會計師二',
       'KAM', 'KAM_len', 'Litigious', 'ModalStrong', 'ModalWeak', 'Negative',
       'Positive', 'Uncertainty'],
      dtype='object')

In [60]:
#round(lstl[0]/word_len[0],4)
inde = ['Litigious','ModalStrong', 'ModalWeak', 'Negative', 'Positive', 'Uncertainty']
lstlp = []
lstmsp = []
lstmwp = []
lstpp = []
lstnp = []
lstucp = []
for i in range(len(lstl)):
    lstlp.append(round(lstl[i]/word_len[i],4))
    lstmsp.append(round(lstms[i]/word_len[i],4))
    lstmwp.append(round(lstmw[i]/word_len[i],4))
    lstpp.append(round(lstp[i]/word_len[i],4))
    lstnp.append(round(lstn[i]/word_len[i],4))
    lstucp.append(round(lstuc[i]/word_len[i],4))

In [61]:
KAM_df["Litigious%"] = lstlp
KAM_df["ModalStrong%"] = lstmsp
KAM_df["ModalWeak%"] = lstmwp
KAM_df["Negative%"] = lstnp
KAM_df["Positive%"] = lstpp
KAM_df["Uncertainty%"] = lstucp

<ipython-input-61-87125ff8af5e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  KAM_df["Litigious%"] = lstlp
<ipython-input-61-87125ff8af5e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  KAM_df["ModalStrong%"] = lstmsp
<ipython-input-61-87125ff8af5e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

####  將第四個檔案 KAM項目 先做處理 取出公司的項目數

KAMi_path = './2020KAMItem_0728.xlsx'
KAMi = pd.read_excel(KAMi_path)
KAMi = pd.DataFrame(KAMi)

In [62]:
KAMi_path = './2021KAMItem_0917.xlsx'
KAMi = pd.read_excel(KAMi_path)
KAMi = pd.DataFrame(KAMi)

In [63]:
KAMi.columns

Index(['Unnamed: 0', '公司代碼', '公司名稱', '年', '次數', 'KAM項次', '證交所產業碼', 'KAM名稱',
       'KAM大類', 'KAM細類', 'KAM細類名稱', 'KAM分類(英國)', '有無提供索引', '有無提供金額',
       '財務報導層級/科目報導層級', '字數', '金額備註', '科目1', '金額1', '佔比類型1', '佔比1', '科目2',
       '金額2', '佔比類型2'],
      dtype='object')

In [66]:
sp = KAMi[['公司代碼', '年', 'KAM項次']]

inde = []
itemc = []

for (index, colname) in enumerate(sp):
    print(index, sp[colname].values)
    if index == 0:
        inde.append(list(sp[colname].values)) #inde[0]
    if index == 2:
        itemc.append(list(sp[colname].values)) #itemc[0]

inre = []
itre = []
for i in range(len(itemc[0])):
    if i<=len(itemc[0])-2 and inde[0][i] == inde[0][i+1]:
        #print(inde[0][i])
        cod = inde[0][i]
        it = itemc[0][i+1]
    else:
        cod = inde[0][i]
        it = itemc[0][i]
        inre.append(cod)
        itre.append(it)

In [67]:
itre_df = pd.DataFrame()
itre_df['公司代碼'] = inre
itre_df['KAM項目數'] = itre
itre_df

,公司代碼,KAM項目數
0,000020,1
1,000022,1
2,000024,1
3,000025,1
4,000026,1
...,...,...
2149,A00025,1
2150,A00037,1
2151,A00041,1
2152,A00047,2


In [68]:
output_df = pd.merge(KAM_df,itre_df, on='公司代碼', how='inner')

In [69]:
avglen = round(output_df['KAM_len']/output_df['KAM項目數'],2)
avglen = avglen.tolist()
output_df['KAM平均詞數'] = avglen

In [70]:
output_df.columns

Index(['公司代碼', '公司名稱', '事務所碼', '事務所名稱', '是否為四大事務所', '四大事務所編碼', '會計師一', '會計師二',
       'KAM', 'KAM_len', 'Litigious', 'ModalStrong', 'ModalWeak', 'Negative',
       'Positive', 'Uncertainty', 'Litigious%', 'ModalStrong%', 'ModalWeak%',
       'Negative%', 'Positive%', 'Uncertainty%', 'KAM項目數', 'KAM平均詞數'],
      dtype='object')

In [71]:
#print(Mas[Mas["公司名稱"].isin(["臺銀證券"])])
#print(Mas[Mas["KAM項次"] > 1])
#print(DF[DF["公司名稱"].isin(["臺銀證券"])])

In [72]:
print('Account_Ori:',len(Account))
print('IndustryCode_Ori:',len(Industry))
print('KAM_Ori:',len(KAM))
print('DF:',len(DF))
print('KAM_df:',len(KAM_df))
print('output_df:',len(output_df))

Account_Ori: 2042
IndustryCode_Ori: 2171
KAM_Ori: 2144
DF: 2025
KAM_df: 2025
output_df: 1997


In [73]:
output_df.to_excel("2021output_DF.xlsx") 